In [2]:
import cv2
import numpy as np

images=[]
images_grey=[]

for i in range(51):
    images.append(cv2.imread(f"C:\\Users\\karan\\hotelImages\\hotel.seq{i}.png"))
    images_grey.append(cv2.imread(f"C:\\Users\\karan\\hotelImages\\hotel.seq{i}.png",0))
images = np.asarray(images)
images_grey = np.asarray(images_grey)


def detectKeypoints (currImage, tau):
    img = np.float64(cv2.cvtColor(currImage,cv2.COLOR_BGR2GRAY))
    Ix = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    Ixx = np.multiply(Ix,Ix)
    Iyy = np.multiply(Iy,Iy)
    Ixy = np.multiply(Ix,Iy)
    Iyx = np.multiply(Ix,Iy)
    Igx = cv2.GaussianBlur(Ixx,(5,5),1)
    Igy = cv2.GaussianBlur(Iyy,(5,5),1)
    Igxy = cv2.GaussianBlur(Ixy,(5,5),1)
    Igyx = cv2.GaussianBlur(Iyx,(5,5),1)
    det = np.multiply(Igx,Igy) - np.multiply(Igxy,Igyx)
    trace =  np.multiply((Igx + Igy),(Igx + Igy))
    Q = det-0.04*(trace)
#     Alpha = 0.04
    final_image,corners = show_corners(Q,img,currImage,tau)
    return corners

def show_corners(corners, img,currImage,tau):
    image = currImage.copy()
    corners = non_maximum_suppresor(corners,img,tau)
    for i in corners:
        x,y = i.ravel()
        image[x][y] = [0,255,0]
    cv2.imshow("Corner", image)
    return image,corners

def non_maximum_suppresor(image, img,tau):
    result = np.zeros((len(img), len(img[0])))
    for i in range(1,len(img) - 1):
        for j in range(1, len(img[0]) - 1):
            if image[i, j] > image[i-1, j-1] and image[i, j] > image[i-1, j+1]\
            and image[i, j] > image[i+1, j-1] and image[i, j] > image[i+1, j+1]\
            and image[i, j] > image[i+1, j] and image[i-1, j] > image[i+1, j+1]\
            and image[i, j] > image[i, j+1] and image[i, j] > image[i, j-1]:
                result[i, j] = image[i, j]
            else:
                result[i, j] = 0
    xmax, xmin = result.max(), result.min()
    result = (result - xmin)/(xmax - xmin)
    HCResponseTreshold = (result >= tau)
    candidates = np.transpose(HCResponseTreshold.nonzero())    
    return candidates

tau = 0.05 # threshold for Harris corner detection
imagesInFolder = 50
currImage = images[0]
results = detectKeypoints(currImage, tau)
cv2.waitKey(0)

-1